# <span style="color: orange;">Preprocessing Démographie et Requêtes Google</span>


### <span style="color: green;">Import des librairies</span>

In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import unicodedata
import glob

import sys
sys.path.append("../src")
from preprocessing import normalize_region_name, get_population_data, consolidate_trends_to_single_sheet, clean_google_trends_data

### <span style="color: green;">Chargement des données</span>

In [9]:
# Données démographiques
filepath = '../data/raw/estim-pop-areg-sexe-gca-1975-2015.xls'
df_pop_train = get_population_data(filepath, 2004, 2011)
df_pop_test = get_population_data(filepath, 2012, 2013)
print(f" Il y a {df_pop_train['region'].nunique()} régions sélectionnées dans les données démographiques.")

Aperçu des données extraites (2004-2011) :


,region,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,year
118,Corse,63276,71661,84706,46314,28161,294118,2006
100,Languedoc-Roussillon,602926,619677,700276,384560,253431,2560870,2007
44,Alsace,449858,489021,527265,236452,140457,1843053,2009
170,Nord - Pas-de-Calais,1127000,1111444,1052694,448892,270518,4010548,2004
124,Lorraine,573475,614496,664908,299472,183398,2335749,2006


Aperçu des données extraites (2012-2013) :


,region,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,year
35,Limousin,151895,156944,203014,130000,96780,738633,2012
28,Centre,613375,585790,695537,403526,265358,2563586,2012
13,Limousin,152739,153814,201303,132833,96820,737509,2013
26,Bourgogne,369529,364039,449307,275565,182690,1641130,2012
22,Alsace,447048,475148,529182,258078,150413,1859869,2012


 Il y a 22 régions sélectionnées dans les données démographiques.


In [10]:
# Données Requêtes Google
# Récupération des fichiers CSV de requêtes Google par région
all_csvs = glob.glob("../data/raw/RequetesGoogleParRegion/*.csv")
print(f" Il y a {len(all_csvs)} fichiers de requêtes Google par région.")

 Il y a 22 fichiers de requêtes Google par région.


### <span style="color: green;">Preprocessing</span>

#### <span style="color: green;">Données démographiques</span>

In [11]:
df_pop_train['region_normalized'] = df_pop_train['region'].apply(normalize_region_name)
df_pop_test['region_normalized'] = df_pop_test['region'].apply(normalize_region_name)
# Vérification
display(df_pop_train[['region', 'region_normalized']].sample(5))

,region,region_normalized
78,Languedoc-Roussillon,LANGUEDOCROUSSILLON
28,Centre,CENTRE
16,Nord - Pas-de-Calais,NORDPASDECALAIS
170,Nord - Pas-de-Calais,NORDPASDECALAIS
80,Lorraine,LORRAINE


In [12]:
# Chemins des fichiers Excel
output_csv_train = '../data/processed/pop_train.csv'
output_csv_test = '../data/processed/pop_test.csv'
# Export vers CSV
df_pop_train.to_csv(output_csv_train, index=False)
df_pop_test.to_csv(output_csv_test, index=False)

#### <span style="color: green;">Données requêtes Google</span>

In [13]:
# Consolidation des données Google Trends en 1 seul fichier Excel
output_excel = '../data/processed/google_trends_consolide.xlsx'
# Création du fichier Excel des requetes avec plusieurs feuilles
with pd.ExcelWriter(output_excel) as writer:
    for file in all_csvs:
        # On récupère le nom de la région depuis le nom du fichier
        region_raw = os.path.splitext(os.path.basename(file))[0]    
        # On normalise ce nom pour la feuille Excel
        sheet_name = normalize_region_name(region_raw)     
        # Lecture du CSV
        # header=2 signifie qu'on utilise la 3ème ligne comme en-tête (on saute les lignes 0 et 1)
        df = pd.read_csv(file, header=1)        
        # Écriture dans la feuille correspondante
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Ajouté : {region_raw} -> Feuille : {sheet_name}")
print(80*'-')
print(f"Le fichier '{output_excel}' a été créé.")

Ajouté : Alsace -> Feuille : ALSACE
Ajouté : Aquitaine -> Feuille : AQUITAINE
Ajouté : Auvergne -> Feuille : AUVERGNE
Ajouté : BasseNormandie -> Feuille : BASSENORMANDIE
Ajouté : Bourgogne -> Feuille : BOURGOGNE
Ajouté : Bretagne -> Feuille : BRETAGNE
Ajouté : CentreValdeLoire -> Feuille : CENTREVALDELOIRE
Ajouté : ChampagneArdenne -> Feuille : CHAMPAGNEARDENNE
Ajouté : Corse -> Feuille : CORSE
Ajouté : FrancheComte -> Feuille : FRANCHECOMTE
Ajouté : HauteNormandie -> Feuille : HAUTENORMANDIE
Ajouté : IledeFrance -> Feuille : ILEDEFRANCE
Ajouté : LanguedocRoussillon -> Feuille : LANGUEDOCROUSSILLON
Ajouté : Limousin -> Feuille : LIMOUSIN
Ajouté : Lorraine -> Feuille : LORRAINE
Ajouté : MidiPyrenees -> Feuille : MIDIPYRENEES
Ajouté : NordPasdeCalais -> Feuille : NORDPASDECALAIS
Ajouté : PaysdelaLoire -> Feuille : PAYSDELALOIRE
Ajouté : Picardie -> Feuille : PICARDIE
Ajouté : PoitouCharentes -> Feuille : POITOUCHARENTES
Ajouté : ProvenceAlpesCotedAzur -> Feuille : PROVENCEALPESCOTEDAZUR


In [14]:
# Cleaning des données Google Trends consolidées
input_excel = '../data/processed/google_trends_consolide.xlsx'
output_excel = '../data/processed/google_trends_clean.xlsx'
clean_google_trends_data(input_excel, output_excel) 

Traitement de 22 feuilles...


KeyError: 'month'

In [15]:
# Consolidation des données Google Trends nettoyées en un seul DataFrame
input_clean = '../data/processed/google_trends_clean.xlsx'
output_final = '../data/processed/google_trends_requetes.xlsx'
df_requetes = consolidate_trends_to_single_sheet(input_clean, output_final)

Lecture du fichier : ../data/processed/google_trends_clean.xlsx
Consolidation de 22 feuilles...
Succès ! Fichier unique créé : ../data/processed/google_trends_requetes.xlsx
Dimensions finales : (3388, 5)


### <span style="color: green;">Export Excel</span>

In [25]:
df_pop_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176 entries, 154 to 11
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   region             176 non-null    object
 1   pop_0_19           176 non-null    object
 2   pop_20_39          176 non-null    object
 3   pop_40_59          176 non-null    object
 4   pop_60_74          176 non-null    object
 5   pop_75_plus        176 non-null    object
 6   pop_total          176 non-null    object
 7   year               176 non-null    int64 
 8   region_normalized  176 non-null    object
dtypes: int64(1), object(8)
memory usage: 13.8+ KB


In [27]:
df_requetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3388 entries, 0 to 3387
Data columns (total 5 columns):
 #   Column                                                                              Non-Null Count  Dtype         
---  ------                                                                              --------------  -----         
 0   Mois                                                                                3388 non-null   datetime64[ns]
 1   region_normalized                                                                   3388 non-null   object        
 2   requete_grippe                                                                      3388 non-null   int64         
 3   requete_grippe_aviaire_vaccin                                                       3388 non-null   int64         
 4   requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie  3388 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory 

In [ ]:
# Fusion left join sur df_train pour garder toutes les lignes de train
df_merged = pd.merge(df_train, df_pop_train, on=['year', 'region_normalized'], how='left')

print("Shape après fusion :", df_merged.shape)
print(df_merged.head())
print(df_merged.info())

Shape après fusion : (9196, 14)
     Id    week  region_code      region_name  TauxGrippe  year  \
0  5523  201152           42           ALSACE          66  2011   
1  5524  201152           72        AQUITAINE          24  2011   
2  5525  201152           83         AUVERGNE          91  2011   
3  5526  201152           25  BASSE-NORMANDIE          49  2011   
4  5527  201152           26        BOURGOGNE          33  2011   

  region_normalized           region pop_0_19 pop_20_39 pop_40_59 pop_60_74  \
0            ALSACE           Alsace   448112    479905    526481    250849   
1         AQUITAINE        Aquitaine   732120    765143    896030    516065   
2          AUVERGNE         Auvergne   294709    305778    375485    223830   
3    BASSENORMANDIE  Basse-Normandie   354514    336201    401766    228888   
4         BOURGOGNE        Bourgogne   372023    369737    450664    269109   

  pop_75_plus pop_total  
0      146978   1852325  
1      344875   3254233  
2      15088